In [ ]:
#Assignment: Sentiment Analysis Happy or Sad

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import nltk
from nltk.corpus import stopwords
import re

# Download stopwords for NLTK
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
data = pd.read_csv('yelp.csv')

In [ ]:
data

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0
...,...,...,...,...,...,...,...,...,...,...
9995,VY_tvNUCCXGXQeSvJl757Q,2012-07-28,Ubyfp2RSDYW0g7Mbr8N3iA,3,First visit...Had lunch here today - used my G...,review,_eqQoPtQ3e3UxLE4faT6ow,1,2,0
9996,EKzMHI1tip8rC1-ZAy64yg,2012-01-18,2XyIOQKbVFb6uXQdJ0RzlQ,4,Should be called house of deliciousness!\n\nI ...,review,ROru4uk5SaYc3rg8IU7SQw,0,0,0
9997,53YGfwmbW73JhFiemNeyzQ,2010-11-16,jyznYkIbpqVmlsZxSDSypA,4,I recently visited Olive and Ivy for business ...,review,gGbN1aKQHMgfQZkqlsuwzg,0,0,0
9998,9SKdOoDHcFoxK5ZtsgHJoA,2012-12-02,5UKq9WQE1qQbJ0DJbc-B6Q,2,My nephew just moved to Scottsdale recently so...,review,0lyVoNazXa20WzUyZPLaQQ,0,0,0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   business_id  10000 non-null  object
 1   date         10000 non-null  object
 2   review_id    10000 non-null  object
 3   stars        10000 non-null  int64 
 4   text         10000 non-null  object
 5   type         10000 non-null  object
 6   user_id      10000 non-null  object
 7   cool         10000 non-null  int64 
 8   useful       10000 non-null  int64 
 9   funny        10000 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 781.4+ KB


In [ ]:
df = data[['text', 'stars']]
df.columns = ['text', 'stars']
df

,text,stars
0,My wife took me here on my birthday for breakf...,5
1,I have no idea why some people give bad review...,5
2,love the gyro plate. Rice is so good and I als...,4
3,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",5
4,General Manager Scott Petello is a good egg!!!...,5
...,...,...
9995,First visit...Had lunch here today - used my G...,3
9996,Should be called house of deliciousness!\n\nI ...,4
9997,I recently visited Olive and Ivy for business ...,4
9998,My nephew just moved to Scottsdale recently so...,2


In [ ]:
df['label'] = df['stars'].apply(lambda x: 'Happy' if x > 3 else 'Sad')

<ipython-input-32-ae6d4f0324b3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['stars'].apply(lambda x: 'Happy' if x > 3 else 'Sad')


In [ ]:
df['label'].value_counts()

,count
label,
Happy,6863
Sad,3137


In [ ]:
#Preprocess the text data
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\W', ' ', text)  # Remove punctuation and special characters
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = re.sub(r'\d', '', text)  # Remove numbers
    stop_words = set(stopwords.words('english'))
    words = text.split()
    text = ' '.join([word for word in words if word not in stop_words])
    return text

In [ ]:
df['text'] = df['text'].apply(preprocess_text)


<ipython-input-39-fa53d304ea3f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(preprocess_text)


In [ ]:
df

,text,stars,label
0,wife took birthday breakfast excellent weather...,5,Happy
1,idea people give bad reviews place goes show p...,5,Happy
2,love gyro plate rice good also dig candy selec...,4,Happy
3,rosie dakota love chaparral dog park convenien...,5,Happy
4,general manager scott petello good egg go deta...,5,Happy
...,...,...,...
9995,first visit lunch today used groupon ordered b...,3,Sad
9996,called house deliciousness could go item item ...,4,Happy
9997,recently visited olive ivy business last week ...,4,Happy
9998,nephew moved scottsdale recently bunch friends...,2,Sad


In [ ]:
#Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)



In [ ]:
#Vectorize the text using TF-IDF (Term Frequency-Inverse Document Frequency)
#vectorizer = TfidfVectorizer(max_features=10000)
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [ ]:
X_test_tfidf

<2000x25831 sparse matrix of type '<class 'numpy.float64'>'
	with 103933 stored elements in Compressed Sparse Row format>

In [ ]:
#Train the Logistic Regression model
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

LogisticRegression()

In [ ]:
print(f"Testing score is {model.score(X_test_tfidf,y_test)} and TrainingScore is {model.score(X_train_tfidf,y_train)} ")

Testing score is 0.824 and TrainingScore is 0.897625 


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score
y_pred= model.predict(X_test_tfidf)
class_names = ['Happy', 'Sad']
print(classification_report(y_test, y_pred, target_names=class_names))

              precision    recall  f1-score   support

       Happy       0.83      0.95      0.88      1403
         Sad       0.81      0.53      0.64       597

    accuracy                           0.82      2000
   macro avg       0.82      0.74      0.76      2000
weighted avg       0.82      0.82      0.81      2000



In [ ]:
user_input = input("Enter an SMS message to classify sentiment: ")
processed_input = preprocess_text(user_input)  # Preprocess the user input
input_tfidf = vectorizer.transform([processed_input])  # Transform the input to the same TF-IDF format
prediction = model.predict(input_tfidf)
prediction

if prediction[0] == 'Happy':
  print("This message is Happy.")
else:
  print("This message is Sad.")


Enter an SMS message to classify sentiment: I am so thrilled
This message is Happy.


In [ ]:
prediction

array(['Happy'], dtype=object)